## 第一階段：取得所有必要的職缺網址

In [77]:
import requests
from bs4 import BeautifulSoup

- 起點：搜尋結果頁面
  - 創造主程式：app.py
  - 創造全域物件：job_urls=[]



In [ ]:
search_url = input('請輸入搜尋頁面網址 ')

In [78]:
search_url = \
'https://www.104.com.tw/jobs/search/?ro=0&keyword=Flask&area=6001002000%2C6001001000&order=1&asc=0&page=1&mode=s&jobsource=2018indexpoc'
print(search_url)

https://www.104.com.tw/jobs/search/?ro=0&keyword=Flask&area=6001002000%2C6001001000&order=1&asc=0&page=1&mode=s&jobsource=2018indexpoc


In [79]:
def url_to_soup(url):
    head = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
        'Accept-Language':'zh-TW,zh;q=0.8,en-US;q=0.6,en;q=0.4'} 
    resp = requests.get(url, headers = head)
    soup = BeautifulSoup(resp.text)
    return soup

In [80]:
def get_page_urls(search_url):
    soup=url_to_soup(search_url)
    jobs=soup.select('.js-job-link')
    page_urls=[]
    for job in jobs:
        page_urls.append('https:'+job['href'])
    return page_urls

In [81]:
import re
import time
import random

In [82]:
def get_all_urls(search_url):
    job_urls=[]
    for page in range(1,11):
        if page > 1:
            if 'page=' in search_url:
                search_url=re.sub(r'page=\d','page='+str(page),search_url)
            else:
                search_url=search_url+'&page=2'
        print(search_url,'處理中')
        page_urls=get_page_urls(search_url)
        print('本頁有',len(page_urls),'個職缺')
        job_urls.extend(page_urls)
        print('已加入job_urls清單')
        if len(page_urls)<20:
            print('本頁為最後一頁')
            break
        time.sleep(random.uniform(5,30))
    else:
        print('共10頁')
    print('總共收集',len(job_urls),'個職缺網址')
    print('完成')
    return job_urls

In [83]:
result=get_all_urls(search_url)

https://www.104.com.tw/jobs/search/?ro=0&keyword=Flask&area=6001002000%2C6001001000&order=1&asc=0&page=1&mode=s&jobsource=2018indexpoc 處理中
本頁有 20 個職缺
已加入job_urls清單
https://www.104.com.tw/jobs/search/?ro=0&keyword=Flask&area=6001002000%2C6001001000&order=1&asc=0&page=2&mode=s&jobsource=2018indexpoc 處理中
本頁有 20 個職缺
已加入job_urls清單
https://www.104.com.tw/jobs/search/?ro=0&keyword=Flask&area=6001002000%2C6001001000&order=1&asc=0&page=3&mode=s&jobsource=2018indexpoc 處理中
本頁有 8 個職缺
已加入job_urls清單
本頁為最後一頁
總共收集 48 個職缺網址
完成


### 將結果寫入JSON檔

In [84]:
from urllib.parse import unquote
import json
from datetime import datetime

#### 先從網址取出關鍵字

In [85]:
keyword=re.search(r'keyword=[^&]*',search_url).group().split('=')[1]

In [86]:
keyword=unquote(keyword)
keyword

'Flask'

#### 記錄當下時間，以指定格式

In [87]:
timestamp=datetime.now().strftime('%m%d-%H%M%S')
timestamp

'0306-231338'

In [88]:
with open(f'job_urls/{keyword}-{timestamp}.json', 'w') as f:
    json.dump(result, f)

In [116]:
len(result)

48

In [89]:
res_set=set(result)

In [90]:
len(res_set)

47

In [91]:
def urls_to_data(result):
    print(f'開始處理，總共有{len(result)}個項目')
    data = []
    for url in result:
        print(url, f'處理中，第{item}項')
        job = {}
        soup = url_to_soup(url)
        job['職位名稱'] = soup.find_all('meta')[4]['content'].rstrip(' - 104人力銀行')
        job['工作地點'] = soup.find('dd', 'addr').text.strip().split()[0]
        job['工作內容'] = soup.find('p').text.replace('\r', '')
        job['擅長工具'] = soup.select('.info')[1].find_all('dd')[-3].text
        job['工作技能'] = soup.select('.info')[1].find_all('dd')[-2].text
        job['其它條件'] = soup.select('.info')[1].find_all('dd')[-1].text.replace(
            '\r', '')
        for k, v in job.items():
            if v in ('不拘', '未填寫'):
                job[k] = ''
        print(job)
        data.append(job)
        time.sleep(random.uniform(5,15))
    print('完成')
    return data

In [127]:
data=urls_to_data(res_set)

https://www.104.com.tw/job/?jobno=6c1h7&jobsource=2018indexpoc 處理中
{'職位名稱': '65240-軟體工程師(台北新生/汐止)_QNAP_威聯通科技股份有限公司', '工作地點': '新北市汐止區中興路22號', '工作內容': '1.軟體開發與測試2.系統分析與設計3.技術研究與開發', '擅長工具': '', '工作技能': '', '其它條件': '經歷: 相關工作經驗 1 年以上 /  相關科系應屆畢業生個性:1. 重視團隊合作2. 樂於分享技術技能:1. 熟悉 Linux Programming (C / Shell Script / GNU Toolchain) 2. 對容器有基本概念 (Docker / Swarm / Kubernetes / LXC / LXD)3. 對虛擬化有基本概念 (QEMU / libvirt)4. 有能力除錯 Ubuntu 桌面系統元件5. 能獨立作業且具良好的問題解決能力，且抱持積極的學習態度者6. 具備 Python 程式及任一 Python Web Framework 使用經驗者佳 (如 Flask/Tornado'}


KeyboardInterrupt: 

In [128]:
data

[{'職位名稱': '65240-軟體工程師(台北新生/汐止)_QNAP_威聯通科技股份有限公司',
  '工作地點': '新北市汐止區中興路22號',
  '工作內容': '1.軟體開發與測試2.系統分析與設計3.技術研究與開發',
  '擅長工具': '',
  '工作技能': '',
  '其它條件': '經歷: 相關工作經驗 1 年以上 /  相關科系應屆畢業生個性:1. 重視團隊合作2. 樂於分享技術技能:1. 熟悉 Linux Programming (C / Shell Script / GNU Toolchain) 2. 對容器有基本概念 (Docker / Swarm / Kubernetes / LXC / LXD)3. 對虛擬化有基本概念 (QEMU / libvirt)4. 有能力除錯 Ubuntu 桌面系統元件5. 能獨立作業且具良好的問題解決能力，且抱持積極的學習態度者6. 具備 Python 程式及任一 Python Web Framework 使用經驗者佳 (如 Flask/Tornado'},
 {'職位名稱': '前端工程師_治略資訊整合股份有限公司',
  '工作地點': '台北市中正區忠孝東路二段25號3樓',
  '工作內容': '工作內容與職責1.熟練Web前端頁面開發技能(HTML/Javascript/CSS)。2.有RESTful API串接經驗。3.不要求美術能力。',
  '擅長工具': 'HTML、JavaScript',
  '工作技能': '',
  '其它條件': '加分項目：1.熟悉uBuntu系統操作。2.具備TDD開發經驗。3.具備React開發經驗。4.具備Flask或Django開發經驗。5.有意願往Full Stack方向學習。  如果願意的話，順便在信件裡回答以下問題，我們會與您回信交流我們想法：為什麼想應徵這個工作？在「必要條件」和「加分項目」中你不符合的項目有哪些？平常都去哪裡看技術文章？你覺得未來幾年內最值得投資的技術是什麼？'},
 {'職位名稱': '數據工程師(Cloud)_勤崴國際科技股份有限公司',
  '工作地點': '台北市中正區羅斯褔路二段100號4樓',
  '工作內容': '1.分析數據資料2.Azure Cloud3.RESTful API',
  '擅長工具':

In [95]:
timestamp=datetime.now().strftime('%m%d-%H%M%S')

with open(f'data/{keyword}-{timestamp}.json', 'w') as f:
    json.dump(data, f)

In [97]:
from collections import Counter

In [119]:
Counter(i['工作地點'][:6] for i in data).most_common()

[('台北市信義區', 10),
 ('台北市內湖區', 9),
 ('台北市松山區', 7),
 ('台北市中正區', 5),
 ('新北市新店區', 4),
 ('台北市南港區', 3),
 ('台北市大安區', 3),
 ('台北市中山區', 3),
 ('新北市汐止區', 1),
 ('新北市三重區', 1),
 ('台北市北投區', 1)]

In [103]:
locations=[]
for d in data:
    locations.append(d['工作地點'][:6])
c=Counter(locations)
c

Counter({'新北市汐止區': 1,
         '台北市中正區': 5,
         '台北市信義區': 10,
         '新北市新店區': 4,
         '台北市松山區': 7,
         '台北市內湖區': 9,
         '台北市南港區': 3,
         '台北市大安區': 3,
         '台北市中山區': 3,
         '新北市三重區': 1,
         '台北市北投區': 1})

In [108]:
sum(dict(c).values())

47

In [109]:
dir(c)

['__add__',
 '__and__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__init__',
 '__init_subclass__',
 '__ior__',
 '__isub__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__missing__',
 '__module__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__weakref__',
 '_keep_positive',
 'clear',
 'copy',
 'elements',
 'fromkeys',
 'get',
 'items',
 'keys',
 'most_common',
 'pop',
 'popitem',
 'setdefault',
 'subtract',
 'update',
 'values']

In [111]:
c.most_common()

[('台北市信義區', 10),
 ('台北市內湖區', 9),
 ('台北市松山區', 7),
 ('台北市中正區', 5),
 ('新北市新店區', 4),
 ('台北市南港區', 3),
 ('台北市大安區', 3),
 ('台北市中山區', 3),
 ('新北市汐止區', 1),
 ('新北市三重區', 1),
 ('台北市北投區', 1)]

In [ ]:
(location for location in 

In [ ]:
https://www.104.com.tw/jobs/search/?ro=0&isnew=3&keyword=kkbox&area=6001002000%2C6001001000&order=1&asc=0&page=1&mode=s&jobsource=2018indexpoc

In [137]:
uuurr=\
'https://www.104.com.tw/jobs/search/?ro=0&isnew=3&keyword=kkbox&area=6001002000%2C6001001000&order=1&asc=0&page=1&mode=s&jobsource=2018indexpoc'

In [138]:
kkbox=get_page_urls(uuurr)

In [139]:
kkbox

['https://www.104.com.tw/job/?jobno=6jbcp&jobsource=2018indexpoc',
 'https://www.104.com.tw/job/?jobno=6jarr&jobsource=2018indexpoc',
 'https://www.104.com.tw/job/?jobno=6jasv&jobsource=2018indexpoc',
 'https://www.104.com.tw/job/?jobno=6jcfj&jobsource=2018indexpoc',
 'https://www.104.com.tw/job/?jobno=6jarc&jobsource=2018indexpoc',
 'https://www.104.com.tw/job/?jobno=6jasj&jobsource=2018indexpoc',
 'https://www.104.com.tw/job/?jobno=6jars&jobsource=2018indexpoc',
 'https://www.104.com.tw/job/?jobno=4aqtr&jobsource=2018indexpoc',
 'https://www.104.com.tw/job/?jobno=6gwn1&jobsource=2018indexpoc',
 'https://www.104.com.tw/job/?jobno=6ft06&jobsource=2018indexpoc',
 'https://www.104.com.tw/job/?jobno=5mgyh&jobsource=joblist_morej',
 'https://www.104.com.tw/job/?jobno=5sye8&jobsource=joblist_morej',
 'https://www.104.com.tw/job/?jobno=4kkx4&jobsource=joblist_morej',
 'https://www.104.com.tw/job/?jobno=6i9jy&jobsource=joblist_morej',
 'https://www.104.com.tw/job/?jobno=6fvc5&jobsource=joblis

In [44]:
random.uniform(1,100)

85.5631939637284

In [15]:
len(job_urls)

88

In [23]:
aa=[1,2,3]

In [20]:
list(aa)

['[', '1', ',', '2', ',', '3', ']']

In [21]:
import json

In [24]:
json.dumps(aa)

'[1, 2, 3]'

In [25]:
with open('data.json', 'w') as f:
    json.dump(aa, f)

In [120]:
uuu='https://www.104.com.tw/job/?jobno=5y8ex'
ppp=url_to_soup(uuu)

In [129]:
ppp

<!DOCTYPE html>
<!--[if lt IE 7]>     <html class="lt-ie9 lt-ie8 lt-ie7"> <![endif]--><!--[if IE 7]>        <html class="lt-ie9 lt-ie8"> <![endif]--><!--[if IE 8]>        <html class="lt-ie9"> <![endif]--><!--[if gt IE 8]><!--><html lang="zh-tw"><!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="no-cache" http-equiv="pragma"/>
<meta content="no-cache" http-equiv="cache-control"/>
<meta content="Backend Developer_帥哥科技有限公司 - 104人力銀行" property="og:title"/>
<meta content="必備技能:
1.In-depth knowledge of Python, NodeJS
2.Experience with Python web frameworks ie. flask/Django/tornado
3.Utilized work queues for background processing
4.In-depth knowledge of Mongo and Redis
5.Excellent understanding of HTTP
6.Experience developing REST APIs
7.Designed for modular systems
8.Worked with a cloud environment
9.Excellent communication and organizational skills" property="og:description"/>
<meta content="https://static.104.com.tw/logo/1

In [130]:
url_1=\
'https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=php&area=6001002000%2C6001001000&order=1&asc=0&page=3&mode=s&jobsource=2018indexpoc'

In [125]:
maps=get_all_urls(url_1)

https://www.104.com.tw/jobs/search/?ro=0&keyword=php&area=6001002000%2C6001001000&order=1&asc=0&page=1&mode=s&jobsource=2018indexpoc 處理中
本頁有 25 個職缺
已加入job_urls清單
https://www.104.com.tw/jobs/search/?ro=0&keyword=php&area=6001002000%2C6001001000&order=1&asc=0&page=2&mode=s&jobsource=2018indexpoc 處理中
本頁有 20 個職缺
已加入job_urls清單
https://www.104.com.tw/jobs/search/?ro=0&keyword=php&area=6001002000%2C6001001000&order=1&asc=0&page=3&mode=s&jobsource=2018indexpoc 處理中
本頁有 20 個職缺
已加入job_urls清單
https://www.104.com.tw/jobs/search/?ro=0&keyword=php&area=6001002000%2C6001001000&order=1&asc=0&page=4&mode=s&jobsource=2018indexpoc 處理中
本頁有 20 個職缺
已加入job_urls清單
https://www.104.com.tw/jobs/search/?ro=0&keyword=php&area=6001002000%2C6001001000&order=1&asc=0&page=5&mode=s&jobsource=2018indexpoc 處理中
本頁有 20 個職缺
已加入job_urls清單
https://www.104.com.tw/jobs/search/?ro=0&keyword=php&area=6001002000%2C6001001000&order=1&asc=0&page=6&mode=s&jobsource=2018indexpoc 處理中
本頁有 20 個職缺
已加入job_urls清單
https://www.104.com.tw/jobs/

In [132]:
page=get_page_urls(url_1)

In [133]:
page

['https://www.104.com.tw/job/?jobno=6f3mm&jobsource=2018indexpoc',
 'https://www.104.com.tw/job/?jobno=6fhfp&jobsource=2018indexpoc',
 'https://www.104.com.tw/job/?jobno=417q4&jobsource=2018indexpoc',
 'https://www.104.com.tw/job/?jobno=5tbum&jobsource=2018indexpoc',
 'https://www.104.com.tw/job/?jobno=6e5wc&jobsource=2018indexpoc',
 'https://www.104.com.tw/job/?jobno=422lq&jobsource=2018indexpoc',
 'https://www.104.com.tw/job/?jobno=6etvb&jobsource=2018indexpoc',
 'https://www.104.com.tw/job/?jobno=68fa6&jobsource=2018indexpoc',
 'https://www.104.com.tw/job/?jobno=5403n&jobsource=2018indexpoc',
 'https://www.104.com.tw/job/?jobno=5duh3&jobsource=2018indexpoc',
 'https://www.104.com.tw/job/?jobno=4jihi&jobsource=2018indexpoc',
 'https://www.104.com.tw/job/?jobno=6fvyh&jobsource=2018indexpoc',
 'https://www.104.com.tw/job/?jobno=5l8ii&jobsource=2018indexpoc',
 'https://www.104.com.tw/job/?jobno=6hylz&jobsource=2018indexpoc',
 'https://www.104.com.tw/job/?jobno=6iwdk&jobsource=2018indexp